# Lab 4: Urban Heat Island Detection using Thermal Satellite Imagery

## Goal
In this exercise, you will:
1. Select and compare two thermal satellite images from Kraków (or another city that has recently undergone intensive, thoughtless concrete development) – one from **2013** and one from **2024**.
2. Each image must be acquired on a **hot summer day** (T > 27°C based on IMGW meteorological data).
3. Images must have **low cloud cover** (< 20%).
4. Visualize thermal data and analyze surface temperature differences (Urban Heat Island effect).

In [2]:
import ee
import geemap
ee.Initialize(project='ee-dabrowaremote')

## Task 1: Area of Interest (AOI)
- Define an AOI over the Reduta Street area in Kraków.
- Use `ee.Geometry.Polygon` or `ee.Geometry.Rectangle`.

In [49]:
aoi = ee.Geometry.Rectangle([19.9762,50.092,19.9889,50.0986]) 

# Visualize the AOI on a map
Map = geemap.Map()
Map.centerObject(aoi, zoom=15)
Map.addLayer(aoi, {'color': 'red'}, 'AOI')
Map

Map(center=[50.095300097386605, 19.982549999987466], controls=(WidgetControl(options=['position', 'transparent…

## Task 2: Download and Analyze IMGW Meteorological Data
- Visit: [IMGW Archive](https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/)
- Navigate to:
  - `dane_meteorologiczne/dobowe/synop/2013/`
  - `dane_meteorologiczne/dobowe/synop/2024/`
- Download `s_d_tmax.csv` for both years.
- Filter the rows for **station ID 12566 (e.g. Kraków-Balice)**.
- Identify days in **July or August** with **TMAX > 27°C**.

In [22]:
import pandas as pd

# Wczytaj dane z pliku CSV
data = pd.read_csv('lab4/k_d_08_2013.csv', delimiter=',', header=None, encoding='cp1250')

# Filtruj dane dla stacji KRAKÓW-OBSERWATORIUM 
station_data = data[data[1] == 'KRAKÓW-OBSERWATORIUM']

# Znajdź datę dnia z największą temperaturą 
max_temp_row = station_data.loc[station_data[5].idxmax()]

# Wyświetl datę i temperaturę
if int(max_temp_row[4]) < 10:
    day = f"0{int(max_temp_row[4])}"
else:
    day = str(int(max_temp_row[4]))

if int(max_temp_row[3]) < 10:
    month = f"0{int(max_temp_row[3])}"
else:
    month = str(int(max_temp_row[3]))

year = str(int(max_temp_row[2]))
max_temp_date_2013 = f"{year}-{month}-{day}"
max_temp_value_2013 = max_temp_row[5]
print(f"Data dnia z największą temperaturą: {max_temp_date_2013}, Temperatura: {max_temp_value_2013}°C")

Data dnia z największą temperaturą: 2013-08-08, Temperatura: 38.3°C


In [23]:
# Wczytaj dane z pliku CSV
data = pd.read_csv('lab4/k_d_08_2024.csv', delimiter=',', header=None, encoding='cp1250')

# Filtruj dane dla stacji KRAKÓW-OBSERWATORIUM 
station_data = data[data[1] == 'KRAKÓW-OBSERWATORIUM          ']

# Znajdź datę dnia z największą temperaturą 
max_temp_row = station_data.loc[station_data[5].idxmax()]

# Wyświetl datę i temperaturę
if int(max_temp_row[4]) < 10:
    day = f"0{int(max_temp_row[4])}"
else:
    day = str(int(max_temp_row[4]))

if int(max_temp_row[3]) < 10:
    month = f"0{int(max_temp_row[3])}"
else:
    month = str(int(max_temp_row[3]))

year = str(int(max_temp_row[2]))
max_temp_date_2024 = f"{year}-{month}-{day}"
max_temp_value_2024 = max_temp_row[5]
print(f"Data dnia z największą temperaturą: {max_temp_date_2024}, Temperatura: {max_temp_value_2024}°C")

Data dnia z największą temperaturą: 2024-08-25, Temperatura: 33.6°C


## Task 3: Select Landsat 8 Images Matching These Dates
- In Earth Engine, use `LANDSAT/LC08/C02/T1_L2` collection.
- Apply filters:
  - `.filterBounds(aoi)`
  - `.filterDate()` for the matching day
  - `.filterMetadata('CLOUD_COVER', 'less_than', 20)`
- Try to find **one image from 2013** and **one image from 2024** that match your hot days list.

In [59]:
#center_date_2013 = ee.Date(max_temp_date_2013)
#center_date_2024 = ee.Date(max_temp_date_2024)

#start_date_2013 = center_date_2013.advance(-15, 'day')
#end_date_2013 = center_date_2024.advance(15, 'day')

#start_date_2024 = center_date_2024.advance(-15, 'day')
#end_date_2024 = center_date_2024.advance(15, 'day')

# Filtruj obrazy Landsat 8 dla 2013
landsat_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2013-08-01', '2013-08-31') \
    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
    .map(lambda img: img.clip(aoi))

# Filtruj obrazy Landsat 8 dla 2024
landsat_2024 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2024-07-01', '2024-08-31') \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER') \
    .map(lambda img: img.clip(aoi))

# Pobierz pierwszy obraz z każdej kolekcji
image_2013 = landsat_2013.first()
image_2024 = landsat_2024.first()

# Dodaj obrazy do mapy
#Map.addLayer(image_2013, {'bands': ['ST_B10'], 'min': 13000, 'max': 16500, 'palette': ['blue', 'green', 'red']}, 'Landsat 8 2013')
#Map.addLayer(image_2024, {'bands': ['ST_B10'], 'min': 13000, 'max': 16500, 'palette': ['blue', 'green', 'red']}, 'Landsat 8 2024')

#Map

## Task 4: Process Thermal Band (ST_B10)
- Convert Band 10 to Brightness Temperature (Kelvin):
  `TB = ST_B10 * 0.00341802 + 149.0`
- Create a visualization of each image using the same color scale.

In [62]:
# Przekształć pasmo ST_B10 na temperaturę jasności (Kelvin)
def calculate_brightness_temperature(image):
    return image.select('ST_B10').multiply(0.00341802).add(149.0).rename('Brightness_Temperature')

# Oblicz temperaturę jasności dla obrazów z 2013 i 2024
tb_2013 = calculate_brightness_temperature(image_2013)
tb_2024 = calculate_brightness_temperature(image_2024)

# Dodaj warstwy do mapy z tą samą skalą kolorów
tb_vis_params = {
    'min': 290,  # Minimalna wartość temperatury w Kelvinach
    'max': 320,  # Maksymalna wartość temperatury w Kelvinach
    'palette': ['blue', 'green', 'yellow', 'red']  # Skala kolorów
}

#Część tasku 5
# Calculate the difference between TB_2024 and TB_2013
tb_difference = tb_2024.subtract(tb_2013).rename('Temperature_Difference')
diff_vis_params = {
    'min': -10,  # Minimum difference value
    'max': 10,   # Maximum difference value
    'palette': ['blue', 'white', 'red']  # Color scale for negative, zero, and positive differences
}

Map2 = geemap.Map()
Map2.centerObject(aoi, zoom=15)
Map2.addLayer(tb_2013, tb_vis_params, '2013')
Map2.addLayer(tb_2024, tb_vis_params, '2024')
Map2.addLayer(tb_difference, diff_vis_params, 'Difference')
# Wyświetl mapę
Map2

Map(center=[50.095300097386605, 19.982549999987466], controls=(WidgetControl(options=['position', 'transparent…

## Task 5: Compare and Interpret
- Compare the two maps.
- Optionally calculate difference: `TB_2024 - TB_2013`
- Discuss: did the surface temperature increase in the area?
- Is there evidence of an Urban Heat Island effect related to development?

### Answer

Because of the lack of data, the results are not reliable. From the data provided by IMGW, we obtained dates of the hottest day. Unfortunetely, the available Landsat images from August were far away from those dates. Moreover, Landsat-8 has a temporal resolution about 16 days, so low number of images is available. Those images are also mostly covered by clouds. As a result, the temperature in 2024 is lower than in 2013 even it should be much higher. Those results should be inverted because of the concrete ground and buildings in 2024 comparing to green areas in 2013. With those data also calculating difference does not change anything.